# Network Analysis with Adjusted Emissions

## Example from social circles data


### Data Description

https://snap.stanford.edu/data/ego-Facebook.html

This dataset (*facebook_combined.txt*) consists of circles from Facebook (4,039 nodes and 88,234 edges) 

### Load required libraries

In [10]:
import networkx as nx
import scipy.sparse as sp
import psutil, time, os, gc, statistics, warnings
import numpy as np
import pandas as pd
from codecarbon import EmissionsTracker

warnings.filterwarnings('ignore')

### Functions

In [11]:
# Function to get memory usage
def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / 1024 / 1024  # Memory in MB

# Function to run analysis with multiple iterations
def run_analysis_with_repeats(analysis_func, file_path, num_runs=5):
    results = []
    
    for _ in range(num_runs):
        gc.collect() # clear memory before each run
        result = analysis_func(file_path)
        results.append(result)
    
    # Compute statistics
    metrics = {
        "Method": results[0]["Method"],
        "Nodes": results[0]["Nodes"],
        "Edges": results[0]["Edges"],
        "Avg Degree": results[0]["Avg Degree"],
        "Density": results[0]["Density"],
        "Time (s)": statistics.mean([r["Time (s)"] for r in results]),
        "Time Std (s)": statistics.stdev([r["Time (s)"] for r in results]) if num_runs > 1 else 0,
        "Memory (MB)": statistics.mean([r["Memory (MB)"] for r in results]),
        "Memory Std (MB)": statistics.stdev([r["Memory (MB)"] for r in results]) if num_runs > 1 else 0,
        "Emissions (kg CO2eq)": statistics.mean([r["Emissions (kg CO2eq)"] for r in results]),
        "Emissions Std (kg CO2eq)": statistics.stdev([r["Emissions (kg CO2eq)"] for r in results]) if num_runs > 1 else 0,
        "Active CPU cores": statistics.mean([r["Active CPU cores"] for r in results]),
        "Total CPU cores": statistics.mean([r["Total CPU cores"] for r in results]),
        "Calibrated Emissions (kg CO2eq)": statistics.mean([r["Calibrated Emissions (kg CO2eq)"] for r in results])
    }
    
    return metrics, results[0]["Top 20 Nodes"]  # Return metrics and top nodes from first run

# Function to analyze full graph with NetworkX
def analyze_networkx(file_path):
    start_time = time.time()
    tracker = EmissionsTracker()
    tracker.start()
    start_memory = get_memory_usage()
    
    G = nx.read_edgelist(file_path, nodetype=int, create_using=nx.Graph())
    
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    avg_degree = sum(dict(G.degree()).values()) / num_nodes
    density = (2 * num_edges) / (num_nodes * (num_nodes - 1)) if num_nodes > 1 else 0
    
    # Find top 20 nodes by degree
    degrees = G.degree()
    top_nodes = sorted(degrees, key=lambda x: x[1], reverse=True)[:20]
    top_nodes_dict = [{"Node": node, "Degree": degree} for node, degree in top_nodes]

    # track CPU usage
    cpu_percents = psutil.cpu_percent(percpu=True)
    active_cores = sum(1 for percent in cpu_percents if percent > 10)
    total_cores = psutil.cpu_count()
    
    exec_time = time.time() - start_time
    memory_used = get_memory_usage() - start_memory
    emissions = tracker.stop()
    calibrated_emissions = emissions * (active_cores / total_cores) if total_cores > 0 else emissions
    
    print(f"Nodes: {num_nodes}")
    print(f"Edges: {num_edges}")
    print(f"Average Degree: {avg_degree:,.2f}")
    print(f"Density: {density:,.6f}")
    print(f"Execution Time: {exec_time:,.2f} seconds")
    print(f"Memory Used: {memory_used:,.2f} MB")
    print(f"Carbon Emissions: {emissions:,.6f} kg CO2eq")
    print(f"Active CPU cores: {active_cores:,.2f}")
    print(f"Total CPU cores: {total_cores:,.2f}")
    print(f"Calibrated Emissions: {calibrated_emissions:,.6f} kg CO2eq")
    print("Top 20 Most Connected Nodes:")
    for node in top_nodes_dict:
        print(f"  Node {node['Node']}: Degree {node['Degree']}")
    
    return {
        "Method": "NetworkX (Full)",
        "Nodes": num_nodes,
        "Edges": num_edges,
        "Avg Degree": avg_degree,
        "Density": density,
        "Time (s)": exec_time,
        "Memory (MB)": memory_used,
        "Emissions (kg CO2eq)": emissions,
        "Active CPU cores": active_cores,
        "Total CPU cores": total_cores,
        "Calibrated Emissions (kg CO2eq)": calibrated_emissions,
        "Top 20 Nodes": top_nodes_dict
    }

# SciPy sparse matrix analysis
def analyze_scipy_sparse(file_path):
    start_time = time.time()   
    tracker = EmissionsTracker()
    tracker.start()
    start_memory = get_memory_usage()
    
    # Load edge list with pandas for better performance
    edges_df = pd.read_csv(file_path, sep=' ', header=None, dtype=np.int32, engine='c')
    edges = edges_df.to_numpy()
    
    # Vectorized node mapping
    nodes, inverse_indices = np.unique(edges, return_inverse=True)
    num_nodes = len(nodes)
    edge_indices = inverse_indices.reshape(edges.shape)  # Shape: (m, 2)
    
    # Create row and column arrays for symmetric adjacency matrix
    rows = np.concatenate([edge_indices[:, 0], edge_indices[:, 1]])
    cols = np.concatenate([edge_indices[:, 1], edge_indices[:, 0]])
    data = np.ones(len(rows), dtype=np.int32)
    
    adj_matrix = sp.csr_matrix((data, (rows, cols)), shape=(num_nodes, num_nodes))
    
    num_edges = adj_matrix.nnz // 2
    degrees = np.array(adj_matrix.sum(axis=1)).flatten()
    avg_degree = degrees.mean()
    density = (2 * num_edges) / (num_nodes * (num_nodes - 1)) if num_nodes > 1 else 0
    
    # Find top 20 nodes by degree
    top_indices = np.argpartition(degrees, -20)[-20:]
    top_degrees = degrees[top_indices]
    top_nodes = [(nodes[i], degrees[i]) for i in top_indices]
    top_nodes = sorted(top_nodes, key=lambda x: x[1], reverse=True)[:20]
    top_nodes_dict = [{"Node": node, "Degree": degree} for node, degree in top_nodes]

    # track CPU usage
    cpu_percents = psutil.cpu_percent(percpu=True)
    active_cores = sum(1 for percent in cpu_percents if percent > 10)
    total_cores = psutil.cpu_count()
    
    exec_time = time.time() - start_time
    memory_used = get_memory_usage() - start_memory
    emissions = tracker.stop()
    calibrated_emissions = emissions * (active_cores / total_cores) if total_cores > 0 else emissions
    
    print(f"Nodes: {num_nodes}")
    print(f"Edges: {num_edges}")
    print(f"Average Degree: {avg_degree:,.2f}")
    print(f"Density: {density:,.6f}")
    print(f"Execution Time: {exec_time:,.2f} seconds")
    print(f"Memory Used: {memory_used:,.2f} MB")
    print(f"Carbon Emissions: {emissions:,.6f} kg CO2eq")
    print(f"Active CPU cores: {active_cores:,.2f}")
    print(f"Total CPU cores: {total_cores:,.2f}")
    print(f"Calibrated Emissions: {calibrated_emissions:,.6f} kg CO2eq")
    print("Top 20 Most Connected Nodes:")
    for node in top_nodes_dict:
        print(f"  Node {node['Node']}: Degree {node['Degree']}")
    
    return {
        "Method": "SciPy Sparse (Full)",
        "Nodes": num_nodes,
        "Edges": num_edges,
        "Avg Degree": avg_degree,
        "Density": density,
        "Time (s)": exec_time,
        "Memory (MB)": memory_used,
        "Emissions (kg CO2eq)": emissions,
        "Active CPU cores": active_cores,
        "Total CPU cores": total_cores,
        "Calibrated Emissions (kg CO2eq)": calibrated_emissions,
        "Top 20 Nodes": top_nodes_dict
    }


### Main function and combined output

In [12]:
# Main execution
def main():
    file_path = "data/facebook_combined.txt"
    
    if not os.path.exists(file_path):
        print(f"Error: {file_path} not found. Please download from http://snap.stanford.edu/data/egnets-Facebook.html")
        return
    
    num_runs = 50
    results = []
    top_nodes_dfs = []
    
    # List of analysis functions to loop over
    analysis_functions = [
        analyze_networkx,
        analyze_scipy_sparse
    ]
    
    # Run each analysis function in a loop
    for analysis_func in analysis_functions:
        print(f"\nRunning {analysis_func.__name__} analysis ({num_runs} runs)")
        metrics, top_nodes = run_analysis_with_repeats(analysis_func, file_path, num_runs)
        results.append(metrics)
        top_nodes_df = pd.DataFrame(top_nodes)
        top_nodes_df['Method'] = metrics["Method"]
        top_nodes_dfs.append(top_nodes_df)

    # Combine results
    print("\nTable: Performance Comparison (Averaged over runs)")
    df = pd.DataFrame(results)
    print(df.round(6))
    df.to_csv("01-03-graph_analysis_comparison.csv", index=False)
    print("\nResults saved to 01-03-graph_analysis_comparison.csv")
    
    # Save top nodes to CSV
    top_nodes_combined = pd.concat(top_nodes_dfs, ignore_index=True)
    top_nodes_combined.to_csv("01-03-top_20_nodes.csv", index=False)
    print("Top 20 nodes saved to 01-03-top_20_nodes.csv")

if __name__ == "__main__":
    main()

[codecarbon WARNING @ 23:04:07] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon WARNING @ 23:04:07] Error while trying to count physical CPUs: [Errno 2] No such file or directory: 'lscpu'. Defaulting to 1.
[codecarbon INFO @ 23:04:07] [setup] RAM Tracking...
[codecarbon INFO @ 23:04:07] [setup] CPU Tracking...
[codecarbon WARNING @ 23:04:07] We saw that you have a Apple M4 but we don't know it. Please contact us.



Running analyze_networkx analysis (50 runs)


[codecarbon WARNING @ 23:04:07] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Mac OS detected: Please install Intel Power Gadget or enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 23:04:07] CPU Model on constant consumption mode: Apple M4
[codecarbon WARNING @ 23:04:07] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 23:04:07] [setup] GPU Tracking...
[codecarbon INFO @ 23:04:07] No GPU found.
[codecarbon INFO @ 23:04:07] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 23:04:07] >>> Tracker's metadata:
[codecarbon INFO @ 23:04:07]   Platform system: macOS-15.5-arm64-arm-64bit-Mach-O
[codecarbon INFO @ 23:04:07]   Python version: 3.13.5
[codecarbon INFO @ 23:04:07]   CodeCarbon version: 3.0.2
[codecarbon INFO @ 23:04:0

Nodes: 4039
Edges: 88234
Average Degree: 43.69
Density: 0.010820
Execution Time: 3.63 seconds
Memory Used: 10.09 MB
Carbon Emissions: 0.000000 kg CO2eq
Active CPU cores: 5.00
Total CPU cores: 10.00
Calibrated Emissions: 0.000000 kg CO2eq
Top 20 Most Connected Nodes:
  Node 107: Degree 1045
  Node 1684: Degree 792
  Node 1912: Degree 755
  Node 3437: Degree 547
  Node 0: Degree 347
  Node 2543: Degree 294
  Node 2347: Degree 291
  Node 1888: Degree 254
  Node 1800: Degree 245
  Node 1663: Degree 235
  Node 1352: Degree 234
  Node 2266: Degree 234
  Node 483: Degree 231
  Node 348: Degree 229
  Node 1730: Degree 226
  Node 1985: Degree 224
  Node 1941: Degree 223
  Node 2233: Degree 222
  Node 2142: Degree 221
  Node 1431: Degree 220


[codecarbon WARNING @ 23:04:11] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Mac OS detected: Please install Intel Power Gadget or enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 23:04:11] CPU Model on constant consumption mode: Apple M4
[codecarbon WARNING @ 23:04:11] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 23:04:11] [setup] GPU Tracking...
[codecarbon INFO @ 23:04:11] No GPU found.
[codecarbon INFO @ 23:04:11] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 23:04:11] >>> Tracker's metadata:
[codecarbon INFO @ 23:04:11]   Platform system: macOS-15.5-arm64-arm-64bit-Mach-O
[codecarbon INFO @ 23:04:11]   Python version: 3.13.5
[codecarbon INFO @ 23:04:11]   CodeCarbon version: 3.0.2
[codecarbon INFO @ 23:04:1

...

Nodes: 4039
Edges: 88234
Average Degree: 43.69
Density: 0.010820
Execution Time: 0.43 seconds
Memory Used: 2.02 MB
Carbon Emissions: 0.000000 kg CO2eq
Active CPU cores: 10.00
Total CPU cores: 10.00
Calibrated Emissions: 0.000000 kg CO2eq
Top 20 Most Connected Nodes:
  Node 107: Degree 1045
  Node 1684: Degree 792
  Node 1912: Degree 755
  Node 3437: Degree 547
  Node 0: Degree 347
  Node 2543: Degree 294
  Node 2347: Degree 291
  Node 1888: Degree 254
  Node 1800: Degree 245
  Node 1663: Degree 235
  Node 1352: Degree 234
  Node 2266: Degree 234
  Node 483: Degree 231
  Node 348: Degree 229
  Node 1730: Degree 226
  Node 1985: Degree 224
  Node 1941: Degree 223
  Node 2233: Degree 222
  Node 2142: Degree 221
  Node 1431: Degree 220


[codecarbon WARNING @ 23:05:16] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Mac OS detected: Please install Intel Power Gadget or enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 23:05:16] CPU Model on constant consumption mode: Apple M4
[codecarbon WARNING @ 23:05:16] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 23:05:16] [setup] GPU Tracking...
[codecarbon INFO @ 23:05:16] No GPU found.
[codecarbon INFO @ 23:05:16] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 23:05:16] >>> Tracker's metadata:
[codecarbon INFO @ 23:05:16]   Platform system: macOS-15.5-arm64-arm-64bit-Mach-O
[codecarbon INFO @ 23:05:16]   Python version: 3.13.5
[codecarbon INFO @ 23:05:16]   CodeCarbon version: 3.0.2
[codecarbon INFO @ 23:05:1

Nodes: 4039
Edges: 88234
Average Degree: 43.69
Density: 0.010820
Execution Time: 0.44 seconds
Memory Used: 3.55 MB
Carbon Emissions: 0.000000 kg CO2eq
Active CPU cores: 10.00
Total CPU cores: 10.00
Calibrated Emissions: 0.000000 kg CO2eq
Top 20 Most Connected Nodes:
  Node 107: Degree 1045
  Node 1684: Degree 792
  Node 1912: Degree 755
  Node 3437: Degree 547
  Node 0: Degree 347
  Node 2543: Degree 294
  Node 2347: Degree 291
  Node 1888: Degree 254
  Node 1800: Degree 245
  Node 1663: Degree 235
  Node 1352: Degree 234
  Node 2266: Degree 234
  Node 483: Degree 231
  Node 348: Degree 229
  Node 1730: Degree 226
  Node 1985: Degree 224
  Node 1941: Degree 223
  Node 2233: Degree 222
  Node 2142: Degree 221
  Node 1431: Degree 220


[codecarbon WARNING @ 23:05:16] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Mac OS detected: Please install Intel Power Gadget or enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 23:05:16] CPU Model on constant consumption mode: Apple M4
[codecarbon WARNING @ 23:05:16] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 23:05:16] [setup] GPU Tracking...
[codecarbon INFO @ 23:05:16] No GPU found.
[codecarbon INFO @ 23:05:16] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 23:05:16] >>> Tracker's metadata:
[codecarbon INFO @ 23:05:16]   Platform system: macOS-15.5-arm64-arm-64bit-Mach-O
[codecarbon INFO @ 23:05:16]   Python version: 3.13.5
[codecarbon INFO @ 23:05:16]   CodeCarbon version: 3.0.2
[codecarbon INFO @ 23:05:1

Nodes: 4039
Edges: 88234
Average Degree: 43.69
Density: 0.010820
Execution Time: 0.43 seconds
Memory Used: 0.00 MB
Carbon Emissions: 0.000000 kg CO2eq
Active CPU cores: 8.00
Total CPU cores: 10.00
Calibrated Emissions: 0.000000 kg CO2eq
Top 20 Most Connected Nodes:
  Node 107: Degree 1045
  Node 1684: Degree 792
  Node 1912: Degree 755
  Node 3437: Degree 547
  Node 0: Degree 347
  Node 2543: Degree 294
  Node 2347: Degree 291
  Node 1888: Degree 254
  Node 1800: Degree 245
  Node 1663: Degree 235
  Node 1352: Degree 234
  Node 2266: Degree 234
  Node 483: Degree 231
  Node 348: Degree 229
  Node 1730: Degree 226
  Node 1985: Degree 224
  Node 1941: Degree 223
  Node 2233: Degree 222
  Node 2142: Degree 221
  Node 1431: Degree 220

Table: Performance Comparison (Averaged over runs)
                Method  Nodes  Edges  Avg Degree  Density  Time (s)  \
0      NetworkX (Full)   4039  88234   43.691013  0.01082  0.691827   
1  SciPy Sparse (Full)   4039  88234   43.691013  0.01082  0.5940

## Reference

1. 